In [258]:
import pandas as pd
salm = pd.read_csv('/Users/evelyn/Desktop/Cubic Spline/salm_nooutliers.csv')

In [293]:
salm

,Unnamed: 0.1,Unnamed: 0,CDCID,GenusName,InitialExposure,LastExposure,TotalCases,Exposurestate,ExposureState,ExposureCounty1,...,PRCP,Median age,Immigrants,RH,SLP-STP,MSXPD-WDSP,Households Median Income,%Food preparation,"%Farming, fishing, and forestry",Population Density
0,0,0,6376,Salmonella,2011-01-08,2011-01-28,11,CT,Connecticut,New London,...,0.037143,39.990137,18789.558900,65.156814,0.447619,6.988889,66931.53973,6.122548,0.181759,410.397085
1,1,2,6917,Salmonella,2011-03-03,2011-03-13,8,CT,Connecticut,New Haven,...,0.166439,38.944932,35658.219180,63.142251,10.584849,7.513636,62243.13425,4.815985,0.186321,1419.958055
2,2,3,7025,Salmonella,2011-03-05,2011-03-05,18,MD,Maryland,Frederick,...,0.000000,38.229863,13284.736990,73.553088,38.650000,12.000000,82495.81370,3.902243,0.245475,350.312225
3,3,4,7031,Salmonella,2011-03-12,2011-03-24,81,RI,Rhode Island,Bristol,...,0.027019,41.007808,11548.093150,66.837121,8.009890,6.838462,62365.46301,6.510218,0.270031,1353.479588
4,4,5,7152,Salmonella,2011-01-06,2011-01-28,6,MI,Michigan,Lenawee,...,0.018696,39.686027,5415.238356,81.461384,31.052174,5.069565,48596.07123,6.053629,1.435769,134.048231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,581,704,296455,Salmonella,2019-08-09,2019-08-17,15,IL,Illinois,Cook,...,0.034000,36.676712,169358.424700,71.251122,22.051852,6.761111,63074.52055,5.495523,0.074659,5515.491582
582,582,705,296585,Salmonella,2019-12-14,2019-12-14,8,TX,Texas,Denton,...,0.000000,35.213973,84453.405480,65.466525,24.750000,7.300000,83540.73699,4.986332,0.120863,920.157616
583,583,707,296745,Salmonella,2019-08-11,2019-09-09,23,OR,Oregon,Lane,...,0.034133,37.646575,37384.545660,58.764847,6.417333,5.702000,58058.94292,6.434262,1.853684,740.973471
584,584,708,296752,Salmonella,2019-08-02,2019-09-28,15,CA,California,San Bernardino,...,0.001380,33.167123,120422.000000,28.991035,60.776840,8.269617,61237.30137,5.565473,0.387235,106.695409


In [259]:
features = ['TEMP','SLP','WDSP','PRCP','RH','SLP-STP','MXSPD-WDSP',
            'Median age','Immigrants','Households Median Income','%Food preparation','%Farming, fishing, and forestry','Population Density']


In [ ]:
# 导入库
import numpy as np
import pandas as pd
data = salm[['TEMP','SLP','WDSP','PRCP','RH','SLP-STP','MSXPD-WDSP',
            'Median age','Immigrants','Households Median Income','%Food preparation','%Farming, fishing, and forestry','Population Density']]

import random
mean = data.mean()
std = data.std()
range_low = mean-3*std
range_high = mean+3*std
new_data = data
num=0
d = []
for i in range(len(salm)):  
    for j in range(len(features)):  
        if range_low[j] > data.iloc[i,j] or data.iloc[i,j] > range_high[j]:
            new_data = new_data.drop([i],axis=0)
            d.append(i)
            num = num+1
            break
data = new_data
for i in range(len(d)):
    salm = salm.drop(d[i],axis=0)
salm.reset_index(drop = True, inplace = True)
salm.to_csv('/Users/evelyn/Desktop/salm_nooutliers.csv')

In [260]:
salm = pd.read_csv('/Users/evelyn/Desktop/salm_nooutliers.csv')

In [261]:
from itertools import combinations
a = []
features = ['TEMP','SLP','WDSP','PRCP','RH','SLP-STP','MSXPD-WDSP',
            'Median age','Immigrants','Households Median Income','%Food preparation','%Farming, fishing, and forestry','Population Density']
for p in combinations(features,9):
    a.append(p)
for p in combinations(features,10):
    a.append(p)
for p in combinations(features,11):
    a.append(p)
for p in combinations(features,12):
    a.append(p)

len(a)

1092

In [262]:
fea = pd.DataFrame()

In [263]:
import numpy as np
from progressbar import ProgressBar
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import xgboost as xgb
import warnings
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
r2 = []
for i in tqdm(range(len(a))):
    fea.loc[i, "Features"] = str(a[i])
    X = salm[list(a[i])]
    y = salm['TotalCases']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 89)
    warnings.filterwarnings("ignore")
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    y_pred_xg = model.predict(X_test)
    fea.loc[i, "xgboost18"] = r2_score(y_test, y_pred_xg)

  0%|          | 0/1092 [00:00<?, ?it/s]

In [264]:
fea['xgboost18'].max()

0.5171013797167611

In [265]:
import numpy as np
from progressbar import ProgressBar
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

for i in tqdm(range(len(a))):
    X = salm[list(a[i])]
    y = salm['TotalCases']
    dt1 = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 89)
    for rm in range(0, 65):
        model2 = DecisionTreeRegressor(random_state = rm)
        model2.fit(X_train, y_train)
        y_pred = model2.predict(X_test)
        dt1.append(r2_score(y_test, y_pred))
    idx_rf = dt1.index(max(dt1))
    fea.loc[i, "DT18"] = max(dt1)
    fea.loc[i, "dt_rs"] = idx_rf   

  0%|          | 0/1092 [00:00<?, ?it/s]

In [266]:
fea['DT18'].max()

0.5764525593362688

In [267]:
import numpy as np
from progressbar import ProgressBar
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

for i in tqdm(range(len(a))):
    X = salm[list(a[i])]
    y = salm['TotalCases']
    rf1 = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 89)
    for rm in range(0,66):
        model2 = RandomForestRegressor(random_state = rm)
        model2.fit(X_train, y_train)
        y_pred = model2.predict(X_test)
        rf1.append(r2_score(y_test, y_pred))
    idx_rf = rf1.index(max(rf1))
    fea.loc[i, "RF18"] = max(rf1)
    fea.loc[i, "rf_rs18"] = idx_rf   

  0%|          | 0/1092 [00:00<?, ?it/s]

In [268]:
fea.to_csv('/Users/evelyn/Desktop/Cubic spline/salm_nooutlier912.csv')

In [274]:
index = fea[fea.xgboost18 == fea["xgboost18"].max()].index.tolist()[0]
print(fea.loc[index,:])
print(fea.loc[index,"Features"])

Features        ('TEMP', 'SLP', 'WDSP', 'PRCP', 'SLP-STP', 'MS...
xgboost18                                                0.517101
DT18                                                     0.481168
dt_rs                                                        30.0
RF18                                                     0.294801
rf_rs18                                                      12.0
rmse_xgboost                                            36.748912
rmse_dt                                                  38.09166
rmse_rf                                                 44.409128
Name: 80, dtype: object
('TEMP', 'SLP', 'WDSP', 'PRCP', 'SLP-STP', 'MSXPD-WDSP', 'Immigrants', 'Households Median Income', '%Food preparation')


In [275]:
index = fea[fea.DT18 == fea["DT18"].max()].index.tolist()[0]
print(fea.loc[index,:])
print(fea.loc[index,"Features"])

Features        ('TEMP', 'WDSP', 'PRCP', 'Median age', 'Immigr...
xgboost18                                                0.245603
DT18                                                     0.576453
dt_rs                                                        59.0
RF18                                                     0.254741
rf_rs18                                                      26.0
rmse_xgboost                                            45.932123
rmse_dt                                                 34.416566
rmse_rf                                                 45.653094
Name: 413, dtype: object
('TEMP', 'WDSP', 'PRCP', 'Median age', 'Immigrants', 'Households Median Income', '%Food preparation', '%Farming, fishing, and forestry', 'Population Density')


In [276]:
index = fea[fea.RF18 == fea["RF18"].max()].index.tolist()[0]
print(fea.loc[index,:])
print(fea.loc[index,"Features"])

Features        ('TEMP', 'SLP', 'WDSP', 'RH', 'SLP-STP', 'MSXP...
xgboost18                                                0.264225
DT18                                                     0.426707
dt_rs                                                        60.0
RF18                                                     0.362781
rf_rs18                                                      57.0
rmse_xgboost                                            45.361659
rmse_dt                                                 40.040975
rmse_rf                                                 42.214416
Name: 139, dtype: object
('TEMP', 'SLP', 'WDSP', 'RH', 'SLP-STP', 'MSXPD-WDSP', 'Immigrants', '%Food preparation', '%Farming, fishing, and forestry')


In [194]:

X = salm[['WDSP','PRCP','SLP-STP','RH','MSXPD-WDSP','Immigrants','%Farming, fishing, and forestry','Population Density']]
y = salm['TotalCases']
dt1 = pd.DataFrame()
for i in tqdm(range(0, 100)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = i)
    for rm in range(0, 65):
        model2 = DecisionTreeRegressor(random_state = rm)
        model2.fit(X_train, y_train)
        y_pred = model2.predict(X_test)
        dt1.loc[i, str(rm)] = r2_score(y_test, y_pred)


  0%|          | 0/100 [00:00<?, ?it/s]

In [272]:
# Calculate the RMSE

import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

for i in tqdm(range(len(fea))):
    X = salm[list(a[i])]
    y = salm['TotalCases']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 89)
    
    #XGBOOST
    warnings.filterwarnings("ignore")
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    y_pred_xg = model.predict(X_test)
    mse = mean_squared_error(y_pred_xg, y_test)
    fea.loc[i, "rmse_xgboost"] = np.sqrt(mse)
    
    #DT
    model2 = DecisionTreeRegressor(random_state = int(fea.loc[i, "dt_rs"]))
    model2.fit(X_train, y_train)
    y_pred_dt = model2.predict(X_test)
    mse = mean_squared_error(y_pred_dt, y_test)
    fea.loc[i, "rmse_dt"] = np.sqrt(mse)
    
    #RF
    model2 = RandomForestRegressor(random_state = int(fea.loc[i, "rf_rs18"]))
    model2.fit(X_train, y_train)
    y_pred_rf = model2.predict(X_test)
    mse = mean_squared_error(y_pred_rf, y_test)
    fea.loc[i, "rmse_rf"] = np.sqrt(mse)

  0%|          | 0/1092 [00:00<?, ?it/s]

In [273]:
fea

,Features,xgboost18,DT18,dt_rs,RF18,rf_rs18,rmse_xgboost,rmse_dt,rmse_rf
0,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.398516,0.461707,48.0,0.294739,16.0,41.013663,38.799463,44.411095
1,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.296658,0.407168,3.0,0.239359,16.0,44.350621,40.717610,46.121800
2,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.452622,0.439647,54.0,0.277565,38.0,39.125515,39.586528,44.948561
3,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.208601,0.416380,39.0,0.275500,26.0,47.045075,40.400013,45.012766
4,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.369012,0.452277,27.0,0.281029,38.0,42.007505,39.137849,44.840674
...,...,...,...,...,...,...,...,...,...
1087,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'SLP-STP', 'MS...",0.289252,0.452135,45.0,0.271376,47.0,44.583531,39.142937,45.140684
1088,"('TEMP', 'SLP', 'WDSP', 'RH', 'SLP-STP', 'MSXP...",0.284880,0.470047,44.0,0.292818,38.0,44.720434,38.497744,44.471534
1089,"('TEMP', 'SLP', 'PRCP', 'RH', 'SLP-STP', 'MSXP...",-0.136230,-0.111888,25.0,-0.038217,27.0,56.370217,55.763119,53.884084
1090,"('TEMP', 'WDSP', 'PRCP', 'RH', 'SLP-STP', 'MSX...",0.298596,0.451153,57.0,0.283875,16.0,44.289498,39.177995,44.751848


In [336]:
# picking model that suprass 50%

df_dt = fea[fea.DT18 > 0.55]
df_xg = fea[fea.xgboost18 > 0.55]
df = pd.concat([df_dt,df_xg])
df.reset_index(inplace = True, drop = True)
df

,Features,xgboost18,DT18,dt_rs,RF18,rf_rs18,rmse_xgboost,rmse_dt,rmse_rf
0,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.248305,0.561047,41.0,0.287868,37.0,45.849785,35.036905,44.626917
1,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'Median age', ...",0.280119,0.554001,38.0,0.267276,47.0,44.869045,35.316967,45.267523
2,"('TEMP', 'SLP', 'WDSP', 'RH', 'SLP-STP', 'Medi...",0.297263,0.564525,26.0,0.296331,9.0,44.331541,34.897793,44.360951
3,"('TEMP', 'SLP', 'WDSP', 'RH', 'SLP-STP', 'Immi...",0.215397,0.551665,47.0,0.311464,38.0,46.842643,35.409350,43.881349
4,"('TEMP', 'WDSP', 'PRCP', 'SLP-STP', 'Median ag...",0.385749,0.552059,34.0,0.267551,38.0,41.446660,35.393790,45.259028
5,"('TEMP', 'WDSP', 'PRCP', 'Median age', 'Immigr...",0.245603,0.576453,59.0,0.254741,26.0,45.932123,34.416566,45.653094
6,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.199791,0.567259,58.0,0.241615,48.0,47.306217,34.788099,46.053359
7,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.198868,0.563498,49.0,0.300835,48.0,47.333492,34.938930,44.218725


In [337]:
features = ['TEMP','SLP','WDSP','PRCP','RH','SLP-STP','MSXPD-WDSP',
            'Median age','Immigrants','Households Median Income','%Food preparation','%Farming, fishing, and forestry','Population Density']
df2 = pd.DataFrame()
for i in range(len(features)):
    df1 = df[df.Features.str.contains(str(features[i]))]
    df2.loc[i, "Features"] = str(features[i])
    df2.loc[i, "times"] = len(df1)
df2

,Features,times
0,TEMP,8.0
1,SLP,7.0
2,WDSP,8.0
3,PRCP,6.0
4,RH,5.0
5,SLP-STP,6.0
6,MSXPD-WDSP,0.0
7,Median age,5.0
8,Immigrants,7.0
9,Households Median Income,8.0


In [307]:
# Shap
import shap
X = salm[['TEMP', 'WDSP', 'PRCP', 'Median age', 'Immigrants', 'Households Median Income', '%Food preparation', '%Farming, fishing, and forestry', 'Population Density']]
y = salm['TotalCases']



clf = DecisionTreeRegressor(random_state = 59)
model = clf.fit(X,y)

# Make prediction on the testing data
explainer = shap.KernelExplainer(model.predict, X)
shap_values = explainer.shap_values(X)



Using 586 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/586 [00:00<?, ?it/s]

In [333]:
shap_values

array([[ 1.42893304e+00, -6.90284008e+00,  5.14789534e-03, ...,
        -1.31594344e-01,  1.78035511e+00, -6.37117666e+00],
       [ 2.05384907e-01,  4.22673222e-02, -4.23235278e-03, ...,
        -1.23586733e-01, -4.79508844e+00,  7.24687483e+00],
       [ 2.57134731e-01, -2.09304404e+00, -3.15740289e-01, ...,
         7.04255038e+00,  1.34080936e-02,  1.41435885e+00],
       ...,
       [-1.08048919e+00, -1.15019706e+00,  2.73240696e-02, ...,
        -9.74341784e-02,  4.04246912e+00, -1.12542865e+00],
       [-6.54782220e-01, -2.78758939e+00,  1.25999512e-01, ...,
        -6.34905737e-01, -4.37450431e+00,  5.42117666e-01],
       [-1.73816634e+00, -3.00766902e+00,  1.24573379e-01, ...,
         7.00199090e-01, -4.25129205e+00, -2.41492158e+00]])

## Stacking

In [ ]:
# stacking
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

test = pd.DataFrame()

for i in tqdm(range(len(a))):
    X = salm[list(a[i])]
    y = salm['TotalCases']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18)
    for j in range(0, 66):
        dt = []
        estimators = [
            ('xg', xgb.XGBRegressor()),
            ('dt', DecisionTreeRegressor(random_state = j))
        ]

        meta_estimator = DecisionTreeRegressor(random_state = j)

        stack = StackingRegressor(estimators=estimators, 
                                    final_estimator=meta_estimator)

        xg.fit(X_train, y_train)

        y_pred = xg.predict(X_test)
        dt.append(r2_score(y_test, y_pred))

    idx_dt = dt.index(max(dt))
    test.loc[i, "DT"] = max(dt)
    test.loc[i, "rs_dt"] = idx_dt

In [254]:
test

,DT,rs_dt
0,0.293058,0.0
1,0.422530,0.0
2,0.374154,0.0
3,0.303680,0.0
4,0.386819,0.0
5,0.430835,0.0
6,0.349185,0.0
7,0.422289,0.0
8,0.356430,0.0
9,0.447189,0.0


In [257]:
fea.head(11)

,Features,xgboost64,xgboost18,DT18,dt_rs,RF18,rf_rs18
0,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.429107,0.293058,0.481403,42.0,0.168415,20.0
1,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.368655,0.422530,0.255589,32.0,0.110383,7.0
2,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.359130,0.374154,0.225514,64.0,0.177397,2.0
3,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.367135,0.303680,-0.057370,31.0,0.154431,9.0
4,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.409742,0.386819,0.329371,25.0,0.139662,18.0
5,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.424967,0.430835,0.199529,63.0,0.177566,36.0
6,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.398460,0.349185,0.502505,45.0,0.225434,48.0
7,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.381812,0.422289,-0.050521,38.0,0.206646,59.0
8,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.418785,0.356430,0.297312,12.0,0.219637,59.0
9,"('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'SLP-STP...",0.409991,0.447189,0.235142,49.0,0.184150,7.0
